In [1]:
import numpy as np
import time
import src.fpga_tracing_func as fpga_tracing_func
import src.web_GUI_config_panel as web_GUI_config_panel
import src.web_GUI_pulse_gen as web_GUI_pulse_gen
import src.data_processing as data_processing
import src.all_of_the_parameters as all_of_the_parameters
import ipywidgets as widgets
import threading
import math
import matplotlib.pyplot as plt
import warnings
from scipy.fftpack import fft,ifft
from scipy.optimize import OptimizeWarning
from scipy.optimize import curve_fit
from pynq import MMIO

"""
FPGA initialization
"""
np.seterr(invalid='ignore')
dma_recv = data_processing.dma_recv
dma_controller_base_address = data_processing.dma.mmio.base_addr
dma_reset_reg = MMIO(dma_controller_base_address + 0x4, 4)

"""
Parameter Setting
"""
point_number_on_screen = all_of_the_parameters.point_number_on_screen
plot_interval = all_of_the_parameters.plot_interval
Nr_Ana_Ch = all_of_the_parameters.Nr_Ana_Ch
Nr_Dig_Ch = all_of_the_parameters.Nr_Dig_Ch
Nr_Ana_Mem = all_of_the_parameters.Nr_Ana_Mem
Nr_Dig_Mem = all_of_the_parameters.Nr_Dig_Mem
Dep_Mem = all_of_the_parameters.Dep_Mem
sampling_time = all_of_the_parameters.sampling_time
conf_dict = all_of_the_parameters.conf_dict
sys_clk_freq = all_of_the_parameters.sys_clk_freq # 100 (MHz)

fpga_tracing_func.fpga_tracing_init(web_GUI_config_panel.GUI_set_nr_samples.value) # initialize tracing and chip configure settings
output_buffer_0 = data_processing.output_buffer_0 # allocate buffers
output_buffer_1 = data_processing.output_buffer_1
mmio = data_processing.mmio
data_processing.mmio_init() # initialize the config_buffer

"""
Start
"""
fpga_tracing_func.enable(1) # enable tracing part

config_fail = 0 # show whether section duration configuration meaningful, 0 successful

def Exp_Start_pg(i, buffer_size):        # connect to CPMG
    
    global output_buffer_0, output_buffer_1 # to modify global buffer objects
    
    np.seterr(invalid='ignore')
    mmio.write(conf_dict['conf_nr_sample'], fpga_tracing_func.osci.read(all_of_the_parameters.fpga_func_dict['C_SET_NR_SAMPLES_CMD']))
    dma_recv.start()
    dma_recv.transfer(output_buffer_0[0:buffer_size]) # dma transfer in unit byte
    fpga_tracing_func.terminate_stream_transfer()
    fpga_tracing_func.start_stream_transfer() # starting stream
    web_GUI_pulse_gen.pg.Exp_Start(1) # start pulse generator    
    try:
        dma_recv.wait()
    except TimeoutError:
        pass
    dma_recv.stop()
    
    output_buffer_1[0:buffer_size] = (i * output_buffer_1[0:buffer_size] + output_buffer_0[0:buffer_size]) / (i + 1) # calculate average based on result from last step
    
    # simulation signal, for debugging without NMR hardware
    # x = np.arange(1,buffer_size + 1)
    # output_buffer_1[0:buffer_size] = 10000 * np.sin(x)/ (x ** 0.1)
    
    points_per_channel = int(buffer_size / 4)
    
    # divide data into 4 channels
    Analog0_stream = output_buffer_1[0:points_per_channel:Nr_Ana_Ch]
    Analog1_stream = output_buffer_1[1:points_per_channel:Nr_Ana_Ch]
    Analog2_stream = output_buffer_1[2:points_per_channel:Nr_Ana_Ch]
    Analog3_stream = output_buffer_1[3:points_per_channel:Nr_Ana_Ch]
    
    # calculate x-axis for time domain signal
    clock_scale = web_GUI_config_panel.GUI_Scale.value
    data_length = len(Analog0_stream) # length of data stream for n packages(echos)
    receiver_time_total = data_length / (sys_clk_freq * (10 ** 6) / clock_scale) # total time length for n echos 
    time_stream = np.linspace(0, receiver_time_total, data_length) # linspace instead of arange  

    ch0_plot, ch1_plot, ch2_plot, ch3_plot, time_stream_sampled = data_processing.func_data_processing_for_plot(time_stream, Analog0_stream, Analog1_stream, Analog2_stream, Analog3_stream, buffer_size / 16)

    func_plot_data(receiver_time_total, time_stream_sampled, ch0_plot, ch1_plot, ch2_plot, ch3_plot) # update plots
    if web_GUI_config_panel.GUI_FFT_run_stop.value == True:
        func_fft_fitting(Analog0_stream, Analog1_stream, Analog2_stream, Analog3_stream)
    else:
        GUI_plot_FFT.update_traces(visible=False, selector=dict(name='T2'))
        GUI_plot_FFT.update_traces(visible=False, selector=dict(name='T2_fit'))
        GUI_plot_FFT.update_traces(visible=False, selector=dict(name='FFT'))
    
    return

def Exp_Avg_pg():
    
    global output_buffer_0, output_buffer_1, buffer_size
    
    mmio.write(conf_dict['conf_exp_stop'], 1) # reset stop flag

    output_buffer_0[:] = 0 # set initial value, no need any more
    output_buffer_1[:] = 0
    
    config_c(1)
    
    total_exp_nr = web_GUI_pulse_gen.left_text.value
    exp_interval = web_GUI_pulse_gen.left_text9.value
    buffer_size = all_of_the_parameters.Nr_Bytes_per_Ch * all_of_the_parameters.Nr_Ana_Ch * fpga_tracing_func.osci.read(all_of_the_parameters.fpga_func_dict['C_SET_NR_SAMPLES_CMD']) \
    * fpga_tracing_func.osci.read(all_of_the_parameters.fpga_func_dict['C_SET_STREAM_NR_RX_PULSE'])
    
    for i in range(total_exp_nr):
        if mmio.read(conf_dict['conf_exp_stop']) == 1:
            web_GUI_config_panel.prog.value = int((i + 1) / total_exp_nr * 100)
            Exp_Start_pg(i, buffer_size) # update output buffer 0
            time.sleep(exp_interval)
        else:
            web_GUI_config_panel.prog.value = 100
            break

    return

def Exp_Start_pg1(i, buffer_size): # connect to FID
    
    np.seterr(invalid='ignore')
    
    global output_buffer_0, output_buffer_1
    
    mmio.write(conf_dict['conf_nr_sample'], fpga_tracing_func.osci.read(all_of_the_parameters.fpga_func_dict['C_SET_NR_SAMPLES_CMD']))
    
    # transfer data from osci through DMA into buffer allocated
    
    dma_recv.start()
    
    dma_recv.transfer(output_buffer_0[0:buffer_size])
    
    fpga_tracing_func.terminate_stream_transfer()
       
    fpga_tracing_func.start_stream_transfer()
    
    web_GUI_pulse_gen.pg_1.Exp_Start(1)
    
    try:
        dma_recv.wait()
    except RuntimeError:
        # print("Time out error.")
        pass

    dma_recv.stop()
    
    fpga_tracing_func.terminate_stream_transfer()
    
    # get averaged signal
    
    output_buffer_1[0:buffer_size] = (i * output_buffer_1[0:buffer_size] + output_buffer_0[0:buffer_size]) / (i + 1) # calculate average based on result from last step
    
    # x = np.arange(1,buffer_size + 1)
    # output_buffer_1[0:buffer_size] = 10000 * np.sin(x)/ (x ** 0.1)    
    
    # divide data into 4 channels
    
    clock_scale = web_GUI_config_panel.GUI_Scale.value
    points_per_channel = int(buffer_size / 4)
    
    Analog0_stream = output_buffer_1[0:points_per_channel:Nr_Ana_Ch]
    Analog1_stream = output_buffer_1[1:points_per_channel:Nr_Ana_Ch]
    Analog2_stream = output_buffer_1[2:points_per_channel:Nr_Ana_Ch]
    Analog3_stream = output_buffer_1[3:points_per_channel:Nr_Ana_Ch]
    
    data_length = len(Analog0_stream) # length of data stream for n packages(echos)
    receiver_time_total = data_length / (sys_clk_freq * (10 ** 6) / clock_scale) # total time length for one channel, one echo for FID
    time_stream = np.linspace(0,receiver_time_total,data_length) # linspace instead of arange before data sampling and compressing for plot 
    
    ch0_plot, ch1_plot, ch2_plot, ch3_plot, time_stream_sampled = data_processing.func_data_processing_for_plot(time_stream, Analog0_stream, Analog1_stream, Analog2_stream, Analog3_stream, buffer_size / 16)

    # calculate x-axis for time domain signal
    
    func_plot_data(receiver_time_total, time_stream_sampled, ch0_plot, ch1_plot, ch2_plot, ch3_plot) # update plots
    if web_GUI_config_panel.GUI_FFT_run_stop.value == True:
        func_fft_processing(Analog0_stream, Analog1_stream, Analog2_stream, Analog3_stream)
    else:
        GUI_plot_FFT.update_traces(visible=False, selector=dict(name='T2'))
        GUI_plot_FFT.update_traces(visible=False, selector=dict(name='T2_fit'))
        GUI_plot_FFT.update_traces(visible=False, selector=dict(name='FFT'))
        
    return

def Exp_Avg_pg1():
    
    global output_buffer_0, output_buffer_1, buffer_size
    
    mmio.write(conf_dict['conf_exp_stop'], 1) # reset stop flag
    
    output_buffer_0[:] = 0
    output_buffer_1[:] = 0
    
    web_GUI_pulse_gen.config_plot_f(1)
    
    total_exp_nr = web_GUI_pulse_gen.left_text6.value
    exp_interval = web_GUI_pulse_gen.left_text9.value
    buffer_size = all_of_the_parameters.Nr_Bytes_per_Ch * all_of_the_parameters.Nr_Ana_Ch * fpga_tracing_func.osci.read(all_of_the_parameters.fpga_func_dict['C_SET_NR_SAMPLES_CMD']) \
    * fpga_tracing_func.osci.read(all_of_the_parameters.fpga_func_dict['C_SET_STREAM_NR_RX_PULSE'])

    for i in range(total_exp_nr):
        if mmio.read(conf_dict['conf_exp_stop']) == 1:
            web_GUI_config_panel.prog.value = int((i + 1) / total_exp_nr * 100)
            Exp_Start_pg1(i, buffer_size) # update output buffer 0
            time.sleep(exp_interval)
        else:
            web_GUI_config_panel.prog.value = 100
            break

    return

def thread_of_Exp_Avg_pg(self): # CPMG
    
    if config_fail == 0 :
        thread_Avg_pg = threading.Thread(target = Exp_Avg_pg)
        thread_Avg_pg.start() 
    else:
        pass


def thread_of_Exp_Avg_pg1(self): # FID
    
    thread_Avg_pg1 = threading.Thread(target = Exp_Avg_pg1)
    thread_Avg_pg1.start()   

def config_c(self):
    
    # rx pulse duration should be longer than the time to transfer the points in a package
    # section duration should not be zero or negative
    # points * echo_nr * 16 should not exceed the length of output_buffer
    global config_fail
    
    config_fail = 0 # if fail, this flag equal to 1
    
    web_GUI_pulse_gen.config_plot_c(self)
    
    clock_scale = web_GUI_config_panel.GUI_Scale.value
    fre_sys = sys_clk_freq * (10 ** 6)
    points_nr = fpga_tracing_func.osci.read(all_of_the_parameters.fpga_func_dict['C_SET_NR_SAMPLES_CMD'])
    
    pulse_delay = web_GUI_pulse_gen.left_text2.value * (10**-6)
    P_180 = web_GUI_pulse_gen.right_text5.value * (10**-6)
    t_echo = web_GUI_pulse_gen.left_text1.value * (10**-6)
    P_90 = web_GUI_pulse_gen.right_text1.value * (10**-6)
    P_RX = web_GUI_pulse_gen.right_text2.value * (10**-6)
    t0 = (t_echo - P_90 - P_180) / 2 - pulse_delay
    t1 = (t_echo - P_RX - P_180) / 2 - pulse_delay * 2
    
    t_package = pulse_delay + P_180 + pulse_delay + t1 + pulse_delay + P_RX + pulse_delay + t1
    
    rest_time = points_nr / (fre_sys / clock_scale) - t_package
    
    if (rest_time < 0) and (web_GUI_pulse_gen.tag_neg_duration == False) and (fpga_tracing_func.osci.read(all_of_the_parameters.fpga_func_dict['C_SET_NR_SAMPLES_CMD']) * 16 * pg.No_echos < len(output_buffer_1)):
        pass
    else:
        f_main.layout.title  = r'CPMG configuration failed(neg duration or dma not idle)! Please reconfigure.'
        config_fail = 1
        print('configure successful')
    
def nmr_stop(self): # terminate the experiment
    
    mmio.write(conf_dict['conf_exp_stop'],0)

def Connect_c(): # connect
    
    web_GUI_pulse_gen.make_box()
    web_GUI_pulse_gen.b_config_c.on_click(config_c) # intText to Object, then to IP core
    web_GUI_pulse_gen.b_start_c.on_click(thread_of_Exp_Avg_pg)
    web_GUI_pulse_gen.b_stop_c.on_click(nmr_stop)
    web_GUI_pulse_gen.b_config_f.on_click(web_GUI_pulse_gen.config_plot_f) # intText to Object, then to IP core
    web_GUI_pulse_gen.b_start_f.on_click(thread_of_Exp_Avg_pg1)
    web_GUI_pulse_gen.b_stop_f.on_click(nmr_stop)
    
    return

# GUI_header
GUI_head = web_GUI_config_panel.GUI_head(web_GUI_config_panel.GUI_logo,web_GUI_config_panel.GUI_logo_iis)

#GUI for NMR_CHIP_CFG
GUI_chip_cfg_box = web_GUI_config_panel.GUI_chip_cfg_box

# Pulse_Gen
Pulse_Gen_Total = widgets.VBox(
#    children = (web_GUI_pulse_gen.sequence_choice_menu_frame,web_GUI_pulse_gen.output_class),
    children = (web_GUI_pulse_gen.output_class,),
    layout = widgets.Layout(
        height='250px', # 240px
        width='1920px',
    ),
)

GUI_pulse_gen = Pulse_Gen_Total
pg,pg1,f_main = web_GUI_pulse_gen.Object_Create(fpga_tracing_func.pulse_gen)  # create instantiation of class
Connect_c()  # function to connect button with the plot function
web_GUI_pulse_gen.sequence_choice_menu.value = 'FID'

# GUI_tracing
GUI_tracing = web_GUI_config_panel.GUI_tracing(
    web_GUI_config_panel.GUI_onoff,
    web_GUI_config_panel.GUI_dig_cbox,
    web_GUI_config_panel.GUI_ana_all,
    web_GUI_config_panel.GUI_zoom1,
    # web_GUI_config_panel.GUI_Enh,
    # web_GUI_config_panel.GUI_bandpass,
    web_GUI_config_panel.GUI_FFT,
    web_GUI_config_panel.GUI_save
)

GUI_config_total = web_GUI_config_panel.GUI_total(GUI_head, GUI_chip_cfg_box, GUI_pulse_gen, GUI_tracing)
web_GUI_config_panel.GUI_prop_init()

"""
Plot Graphic
"""

GUI_plot = web_GUI_config_panel.GUI_plot()
GUI_plot_FFT = web_GUI_config_panel.GUI_plot_FFT()
GUI_plot_total = widgets.HBox(
        [
            GUI_plot,
            GUI_plot_FFT,
        ],
        layout = widgets.Layout(
            height='370px',
            width='1298px', # 1680 original
            border='solid 4px cyan',
            margin='0px 5px 0px 5px',
            padding='0px 0px 0px 0px'
        ),
        align_items='center',
    )


"""
display GUI
"""

GUI_total = widgets.VBox(
    [GUI_config_total, GUI_plot_total],
    layout = widgets.Layout(height='100%', width='100%'),)

display(GUI_total)

"""
Functions for Data Processing
"""

# sampling FFT

def find_maxfft(data): # function for acquiring maximum amplitude within one echo
    fft_y = fft(data) #fast fourier transformation
    abs_y = np.abs(fft_y) # choose the abs from complex number, Bilateral spectrum(双边频谱)
    max_fft = max(abs_y)
    return max_fft

def func(t, s0, T2, c): # T2 relaxation fitting paradigm
    return s0 * np.exp(-t/T2) + c

def func_fft_fitting(Analog0_stream, Analog1_stream, Analog2_stream, Analog3_stream): # for CPMG T2 relaxation
    
    # neglect optimizaWarning from scipy
    warnings.simplefilter("ignore", OptimizeWarning)
    
    GUI_plot_FFT.update_annotations(visible = True) # show annotation
    points_per_pkg = fpga_tracing_func.osci.read(all_of_the_parameters.fpga_func_dict['C_SET_NR_SAMPLES_CMD'])
    nr_rx = fpga_tracing_func.osci.read(all_of_the_parameters.fpga_func_dict['C_SET_STREAM_NR_RX_PULSE'])
    
    # preprocessing
    fft_ch = mmio.read(conf_dict['conf_fft_ch'])
    fft_onoff = mmio.read(conf_dict['conf_fft_onoff'])
    data = [0]
    
    if (fft_ch == 0 and fft_onoff == 1): # onoff为FFT_RUNSTOP按钮，ch为选择菜单
        data = Analog0_stream

    elif (fft_ch == 1 and fft_onoff == 1):
        data = Analog1_stream

    elif (fft_ch == 2 and fft_onoff == 1):
        data = Analog2_stream

    elif (fft_ch == 3 and fft_onoff == 1):
        data = Analog3_stream
        
    # processing
    if nr_rx < 3: # difficult for plot
        return
    
    if fft_onoff == 1:
        max_vec = []
        for i in range(0,nr_rx):
            in_data = data[0 + i * points_per_pkg : (i + 1) * points_per_pkg -1]
            max_echo_fft = find_maxfft(in_data)
            max_vec.append(max_echo_fft)
        clock_scale = web_GUI_config_panel.GUI_Scale.value
        fs = sys_clk_freq * (10 ** 6) / clock_scale
        xdata = np.linspace(0, nr_rx - 1, nr_rx) * pg.t_echo / (10 ** 8) # echo time as interval between echos
        ydata = max_vec / max(max_vec)
        # neglect optimizaWarning from scipy
        warnings.simplefilter("ignore", OptimizeWarning)

        try:
            # try fitting
            popt, pcov = curve_fit(func, xdata, ydata)
        except ValueError:
            # if NaN or Inf, pass
            print("Value Error!")
            return
        else:
            # print("Other Error!")
            pass
            
        GUI_plot_FFT.update_xaxes(title_text='Time(s)')
        GUI_plot_FFT.update_xaxes(range=[0, xdata.max()])
        GUI_plot_FFT.update_yaxes(range=[-0.1, 1.1])
        GUI_plot_FFT.update_traces(visible=False, selector=dict(name='FFT'))
        y_fit = func(xdata, *popt)
        y_fit = np.array(y_fit)
        GUI_plot_FFT.update_traces(
        x = xdata,
        y = y_fit,
        visible = True,
        selector = ({'name':'T2_fit'})
        )
        GUI_plot_FFT.update_traces(
        x = xdata,
        y = ydata,
        visible = True,
        selector = ({'name':'T2'})
        )
        
        x_annotation = xdata.max() * 2 / 3
        new_annotation = dict(text = 'T2 = %d' % popt[1], x = x_annotation, y = 0.95)
        GUI_plot_FFT.update_layout(annotations=[dict(xref='x', yref='y', showarrow=False, **new_annotation)])
        
    return

def func_fft_processing(Analog0_stream, Analog1_stream, Analog2_stream, Analog3_stream): # for FID
    
    # preprocessing
    fft_ch = mmio.read(conf_dict['conf_fft_ch'])
    fft_onoff = mmio.read(conf_dict['conf_fft_onoff'])
    data = [0]
    
    if (fft_ch == 0 and fft_onoff == 1): # onoff为FFT_RUNSTOP按钮，ch为选择菜单
        data = Analog0_stream

    elif (fft_ch == 1 and fft_onoff == 1):
        data = Analog1_stream

    elif (fft_ch == 2 and fft_onoff == 1):
        data = Analog2_stream

    elif (fft_ch == 3 and fft_onoff == 1):
        data = Analog3_stream
    
    # data = data[0::all_of_the_parameters.plot_interval] # if interval applies

    # preprocessing: get rid of DC component
    mean = np.mean(data)
    data = np.subtract(data, mean.astype(np.int32))
    
    # fast fourier transformation
    fft_y = fft(data)
    
    abs_y = np.abs(fft_y) # choose the abs from complex number, Bilateral spectrum(双边频谱)
    max_amplitude = max(abs_y)
    normalization_y = abs_y / max_amplitude # Normalization process (bilateral spectrum) 归一化处理（双边频谱）                              
    normalization_half_y = normalization_y[range(int(len(data)/2))] # Due to symmetry, only half of the interval (one-sided spectrum) is taken由于对称性，只取一半区间（单边频谱）
    
    func_plot_FFT(normalization_half_y)   
    
    return

def func_plot_FFT(input_data):

    GUI_plot_FFT.layout.yaxis.range = [0, 1]
    GUI_plot_FFT.update_xaxes(title_text='Frequency(Hz)')
    GUI_plot_FFT.update_traces(visible=False, selector=dict(name='T2'))
    GUI_plot_FFT.update_traces(visible=False, selector=dict(name='T2_fit'))
    GUI_plot_FFT.update_annotations(visible=False)
    
    clock_scale = web_GUI_config_panel.GUI_Scale.value
    data_length = fpga_tracing_func.osci.read(all_of_the_parameters.fpga_func_dict['C_SET_NR_SAMPLES_CMD'])
    fs = sys_clk_freq * (10 ** 6) / clock_scale # sampling rate after CIC
    receiver_time_total = data_length / fs

    N = len(input_data) * 2 # length of input data
    try:
        f_fft = np.arange(0, N) * (fs/N) # x-axis of Fourier Spectrum
    except ZeroDivisionError:
        print("An ZeroDivision Error occurred.")
        return
    except:
        print("An other error occurred.")
        return
    GUI_plot_FFT.update_traces(
    x = f_fft,
    y = input_data,
    visible = True,
    selector = ({'name':'FFT'})
    )
    GUI_plot_FFT.update_xaxes(range=[0, f_fft[-1]])
    
    return

def func_plot_data(x_axis_range, time_stream, ch0, ch1, ch2, ch3):
    
    GUI_plot.update_layout(xaxis_range=[0, x_axis_range]) # update x-axis range first
    
    code_adc = 4.1 / (2 ** 16) * 1000 # digital value to mV
    
    ch0_volts = ch0 * code_adc
    ch1_volts = ch1 * code_adc
    ch2_volts = ch2 * code_adc
    ch3_volts = ch3 * code_adc
    
    GUI_plot.update_traces(
        x = time_stream,
        y = ch0_volts,
        selector = ({'name':'Channel 0'})
    )
    GUI_plot.update_traces(
        x = time_stream,
        y = ch1_volts,
        selector = ({'name':'Channel 1'})
    )
    GUI_plot.update_traces(
        x = time_stream,
        y = ch2_volts,
        selector = ({'name':'Channel 2'})
    )    
    GUI_plot.update_traces(
        x = time_stream,
        y = ch3_volts,
        selector = ({'name':'Channel 3'})
    )
        
    return


def func_system_cycle(): # give all the 2 ** 15 samples out
    
    # parameter loading
    trigger  = web_GUI_config_panel.GUI_TBtn_arm.value
#     enh_sen  = mmio.read(conf_dict['conf_enh_sen'])
#     enh_res  = mmio.read(conf_dict['conf_enh_res'])
#     dc_onoff = mmio.read(conf_dict['conf_enh_dc'])
#     enh_ch   = mmio.read(conf_dict['conf_enh_ch'])

    if dma_recv.idle == True:
        buffer_size = all_of_the_parameters.Nr_Bytes_per_Ch * all_of_the_parameters.Nr_Ana_Ch * fpga_tracing_func.osci.read(all_of_the_parameters.fpga_func_dict['C_SET_NR_SAMPLES_CMD']) * fpga_tracing_func.osci.read(all_of_the_parameters.fpga_func_dict['C_SET_STREAM_NR_RX_PULSE'])
        mmio.write(conf_dict['conf_nr_sample'], fpga_tracing_func.osci.read(all_of_the_parameters.fpga_func_dict['C_SET_NR_SAMPLES_CMD']))
        Analog0_stream, Analog1_stream, Analog2_stream, Analog3_stream = data_processing.func_update_data(fpga_tracing_func.osci.read(all_of_the_parameters.fpga_func_dict['C_SET_STREAM_NR_RX_PULSE']), buffer_size) # func_update_data用于接收包至output_buffer_0
        thread_fft = threading.Thread(target=func_fft_processing, args=(Analog0_stream, Analog1_stream, Analog2_stream, Analog3_stream))
        thread_fft.start()

        if (trigger == 1): #0 = not triggered, 1 = triggered
            web_GUI_config_panel.GUI_TBtn_run_stop.value = False
            web_GUI_config_panel.GUI_TBtn_run_stop.description='Single'
            web_GUI_config_panel.GUI_TBtn_run_stop.button_style = 'warning'
        else:
            web_GUI_config_panel.GUI_TBtn_run_stop.description='Run/Stop'

#         if (dc_onoff == 1):
#             if   (enh_ch == 0):
#                 Analog0_stream = data_processing.func_dc_remove(Analog0_stream)
#             elif (enh_ch == 1):
#                 Analog1_stream = data_processing.func_dc_remove(Analog1_stream)
#             elif (enh_ch == 2):
#                 Analog2_stream = data_processing.func_dc_remove(Analog2_stream)
#             elif (enh_ch == 3):
#                 Analog3_stream = data_processing.func_dc_remove(Analog3_stream)
        
        #nr_of_samples =  fpga_tracing_func.osci.read(all_of_the_parameters.fpga_func_dict['C_SET_NR_SAMPLES_CMD'])
        #nr_of_rx_pulse = fpga_tracing_func.osci.read(all_of_the_parameters.fpga_func_dict['C_SET_STREAM_NR_RX_PULSE'])
        # samples_total =  nr_of_samples * nr_of_rx_pulse
        ch0_plot, ch1_plot, ch2_plot, ch3_plot = data_processing.func_data_processing_for_plot(Analog0_stream, Analog1_stream, Analog2_stream, Analog3_stream, all_of_the_parameters.plot_interval)

        func_plot_data(ch0_plot, ch1_plot, ch2_plot, ch3_plot)

        thread_fft.join()
    else:
        pass
    time.sleep(0.05)

    return Analog0_stream, Analog1_stream, Analog2_stream, Analog3_stream

def thread_of_plot_data():
    
    global output_buffer_0, output_buffer_1
    
    while True:
        
        trigger = web_GUI_config_panel.GUI_TBtn_arm.value
        if mmio.read(conf_dict['conf_onoff']) == 1:
            if mmio.read(conf_dict['conf_token']) == 0:
                Analog0_stream, Analog1_stream, Analog2_stream, Analog3_stream = func_system_cycle()
            else:
                data_processing.func_update_config_tracing()
                Analog0_stream, Analog1_stream, Analog2_stream, Analog3_stream = func_system_cycle()
            web_GUI_config_panel.GUI_save_button.disabled = False
            if (trigger == 1): # 0 = not triggered, 1 = triggered
                web_GUI_config_panel.GUI_TBtn_run_stop.value = False
        else:
            # time.sleep(0.1)
            pass
        if mmio.read(conf_dict['conf_save_file']) == 1:
            buffer_size = all_of_the_parameters.Nr_Ana_Ch * fpga_tracing_func.osci.read(all_of_the_parameters.fpga_func_dict['C_SET_NR_SAMPLES_CMD']) * fpga_tracing_func.osci.read(all_of_the_parameters.fpga_func_dict['C_SET_STREAM_NR_RX_PULSE'])
            try:
                data_processing.func_save_file(output_buffer_1[0:buffer_size:Nr_Ana_Ch], output_buffer_1[1:buffer_size:Nr_Ana_Ch], output_buffer_1[2:buffer_size:Nr_Ana_Ch], output_buffer_1[3:buffer_size:Nr_Ana_Ch])
            except (UnboundLocalError, NameError):
                print("Error when saving file.")
                pass
        mmio.write(conf_dict['conf_save_file'], 0)
    return

buffer_size = all_of_the_parameters.Nr_Ana_Ch * fpga_tracing_func.osci.read(all_of_the_parameters.fpga_func_dict['C_SET_NR_SAMPLES_CMD']) * fpga_tracing_func.osci.read(all_of_the_parameters.fpga_func_dict['C_SET_STREAM_NR_RX_PULSE'])
Thread_plot = threading.Thread(target = thread_of_plot_data)
Thread_plot.start()